# Import Libraries

In [1]:
import numpy as np #numerical computation
import pandas as pd #data wrangling
import matplotlib.pyplot as plt #plotting package

# Import Dataset

In [2]:
df = pd.read_csv("D:\\technocalobs\\Bigmart\\train_modified.csv")
df.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Outlet_Years,Item_Fat_Content_1,Item_Fat_Content_2,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_1,...,Item_Type_Combined_2,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,9.30,0.931078,249.8092,3735.1380,14,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,5.92,0.933420,48.2692,443.4228,4,0,1,0,1,1,...,0,0,0,1,0,0,0,0,0,0
2,17.50,0.872790,141.6180,2097.2700,14,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,19.20,0.818182,182.0950,732.3800,15,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,8.93,0.750000,53.8614,994.7052,26,1,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0


Created a list remove_cols to remove those columns which doesn't required for Model Building

# Data Preprocessing

In [4]:
y = df.Item_Outlet_Sales.values
X = df.drop('Item_Outlet_Sales',axis = 1)

In [5]:
print(X.shape,y.shape)

(8523, 24) (8523,)


Trained a Random Forest model

In [6]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

List of parameters for hyperparameter tuning

In [7]:
param = {
    'max_depth':[3,6,9,12],
    'n_estimators' : [10,50,100,200] 
}

# Hyperparameter optimization using RandomizedSearchCV

In [8]:
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(rf,param_distributions=param,n_iter=5,scoring=make_scorer(mean_squared_error),n_jobs=-1,cv=5,verbose=3)

In [9]:
random_search.fit(X,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=5, n_jobs=-1,
                   param_distributions={'max_depth': [3, 6, 9, 12],
                                        'n_estimators': [10, 50, 100, 200]},
                   scoring=make_scorer(mean_squared_error), verbose=3)

In [10]:
means = random_search.cv_results_['mean_test_score']
params = random_search.cv_results_['params']
for mean, param in zip(means, params):
    print("%f with: %r" % (mean, param))
    if mean == min(means):
        print('Best parameters with the minimum Mean Square Error are:',param)

1193391.871424 with: {'n_estimators': 200, 'max_depth': 6}
Best parameters with the minimum Mean Square Error are: {'n_estimators': 200, 'max_depth': 6}
1278107.913542 with: {'n_estimators': 10, 'max_depth': 12}
1201919.716959 with: {'n_estimators': 100, 'max_depth': 9}
1202076.964129 with: {'n_estimators': 200, 'max_depth': 9}
1227605.873357 with: {'n_estimators': 200, 'max_depth': 12}


Evaluating the model for Train and Test set

In [11]:
rf = RandomForestRegressor(
    n_estimators=100, max_depth=6,
)

In [12]:
rf.fit(X,y)

RandomForestRegressor(max_depth=6)

In [13]:
y_pred = rf.predict(X)

In [14]:
from sklearn.metrics import r2_score,mean_squared_error

score = r2_score(y,y_pred)
print("Score:",100*score)
print("RMSE : %.4g" % np.sqrt(mean_squared_error(y,y_pred)))

Score: 61.47039812559252
RMSE : 1059


In [ ]:
from sklearn.metrics import mean_squared_error,make_scorer,mean_absolute_error
from sklearn.model_selection import cross_val_score

#Perform cross-validation:
cv_score = cross_val_score(rf,X, y, cv=20, scoring = make_scorer(mean_squared_error))
cv_score = np.sqrt(np.abs(cv_score))
    
#Print model report:
print("\nModel Report")
print("MAE : %.4g" % np.sqrt(mean_absolute_error(y,y_pred)))
print("RMSE : %.4g" % np.sqrt(mean_squared_error(y,y_pred)))
print("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
